# Cloud Functions 

In [ ]:
!pip install google-cloud-functions google-auth google-cloud-secret-manager


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [google-cloud-functions]


In [12]:
!pip install google-cloud-functions google-auth google-cloud-storage


In [29]:
import os
from google.cloud import storage
from google.cloud import functions_v2
from google.cloud.functions_v2.types import Function, BuildConfig, CreateFunctionRequest
from google.protobuf import duration_pb2
import zipfile



In [30]:
os.environ["GOOGLE_CLOUD_PROJECT"] = "acecloud-sel-1"  # reemplaza con tu proyecto
project_id = os.environ["GOOGLE_CLOUD_PROJECT"]
location = "us-central1"  # región donde desplegar la función
function_name = "saludo_profesor"  # nombre de la función
bucket_name = "cloudfunc"  # reemplaza con tu bucket de GCS
zip_filename = "funcion.zip"



In [31]:
# Crear cliente
storage_client = storage.Client()

# Verificar si el bucket ya existe
bucket = storage_client.bucket(bucket_name)
if not bucket.exists():
    bucket = storage_client.create_bucket(bucket_name, location="US")
    print(f"✅ Bucket '{bucket_name}' creado correctamente.")
else:
    print(f"⚠️ Bucket '{bucket_name}' ya existe.")

⚠️ Bucket 'cloudfunc' ya existe.


In [33]:
# Crear cliente de Cloud Functions
client = functions_v2.CloudFunctionsServiceClient()

# Definir la función en Python
# Guardamos el código de la función en un archivo local
func_code = """
def saludo(request):
    name = request.args.get('name') or 'estudiante'
    return f'¡Hola, {name}! Bienvenido a Google Cloud Functions 😊'
"""

AttributeError: module 'google.cloud.functions_v2' has no attribute 'CloudFunctionsServiceClient'

In [6]:
with open("main.py", "w") as f:
    f.write(func_code)

# También necesitamos un requirements.txt
with open("requirements.txt", "w") as f:
    f.write("")  # sin dependencias externas para este ejemplo


In [19]:
# ---------------------------
# 3️⃣ Empaquetar en zip
# ---------------------------
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write("main.py")
    zipf.write("requirements.txt")

print("✅ Función empaquetada en funcion.zip")

# ----------------------------
# 4️⃣ Subir zip a GCS
# ---------------------------
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(zip_filename)
blob.upload_from_filename(zip_filename)
print(f"✅ Zip subido a gs://{bucket_name}/{zip_filename}")

# ---------------------------
# 5️⃣ Crear objeto Function usando storage_source
# ---------------------------
parent = f"projects/{project_id}/locations/{location}"

function = Function(
    name=f"{parent}/functions/{function_name}",
    build_config=BuildConfig(
        runtime="python310",
        entry_point="saludo",
        source={"storage_source": {"bucket": bucket_name, "object": zip_filename}}
    )
)


✅ Función empaquetada en funcion.zip
✅ Zip subido a gs://cloudfunc/funcion.zip


In [40]:
import os
from google.cloud import storage
from google.cloud import functions_v2
from google.cloud.functions_v2.types import Function, BuildConfig, Source, CreateFunctionRequest
import zipfile
from google.cloud.functions_v2.types import StorageSource


# ---------------------------
# 1️⃣ Empaquetar función en zip
# ---------------------------
func_code = """
def saludo(request):
    name = request.args.get('name') or 'estudiante'
    return f'¡Hola, {name}! Bienvenido a Google Cloud Functions 😊'
"""

with open("main.py", "w") as f:
    f.write(func_code)

zip_filename = "funcion.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write("main.py")
print("✅ Función empaquetada en funcion.zip")

# ---------------------------
# 2️⃣ Subir zip a GCS
# ---------------------------
bucket_name = "cloudfunc"  # reemplaza con tu bucket de GCS
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(zip_filename)
blob.upload_from_filename(zip_filename)
print(f"✅ Zip subido a gs://{bucket_name}/{zip_filename}")



✅ Función empaquetada en funcion.zip
✅ Zip subido a gs://cloudfunc/funcion.zip


In [1]:

from google.cloud import functions_v2
from google.cloud.functions_v2.types import Function, BuildConfig, Source, StorageSource, CreateFunctionRequest

# Datos
project_id = "acecloud-sel-1"
location = "us-central1"
function_name = "saludoprofesor"  # Este será el function_id
bucket_name = "cloudfunc"  # reemplaza con tu bucket de GCS
zip_filename = "funcion.zip"

parent = f"projects/acecloud-sel-1/locations/us-central1"

# Creamos la función SIN definir 'name'
function = Function(
    build_config=BuildConfig(
        runtime="python310",
        entry_point="saludo",
        source=Source(
            storage_source=StorageSource(
                bucket=bucket_name,
                object_=zip_filename
            )
        )
    )
)

# Cliente v2
client = functions_v2.FunctionServiceClient()

# Se pasa el function_id separado
request = CreateFunctionRequest(
    parent=parent,
    function=function,
    function_id=function_name
)

# Desplegar función
operation = client.create_function(request=request)
print("⏳ Desplegando función... Esto puede tardar unos minutos.")
operation.result()
print(f"✅ Función {function_name} desplegada correctamente en {location}")



InvalidArgument: 400 Invalid resource name '' for pattern 'projects/{project=*}/locations/{location=*}/functions/{function=*}'.

In [5]:
pip show google-cloud-functions


Name: google-cloud-functions
Version: 1.21.0
Summary: Google Cloud Functions API client library
Home-page: https://github.com/googleapis/google-cloud-python/tree/main/packages/google-cloud-functions
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /Users/adrianacamposnarvaez/anaconda3/envs/bq/lib/python3.11/site-packages
Requires: google-api-core, google-auth, grpc-google-iam-v1, grpcio, proto-plus, protobuf
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade google-cloud-functions


Note: you may need to restart the kernel to use updated packages.


In [4]:
from google.cloud import functions_v2
from google.cloud.functions_v2.types import Function, BuildConfig, Source, StorageSource, CreateFunctionRequest
client = functions_v2.FunctionServiceClient()
